In [ ]:
!pip install langchain openai pinecone-client langchain-community sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5

In [ ]:
# 벡터 DB 및 llm 라이브러리
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
import openai

# 임베딩 모델 위한 라이브러리
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import numpy as np

In [ ]:
# 1. OpenAI API 키 설정 및 임베딩 모델 초기화
openai_api_key = "your-api-key"

In [ ]:
# multi-qa-mpnet-base-dot-v1 모델 로드
#model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')   # 더 경량화된 모델

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 2. 파인콘 초기화
from pinecone import Pinecone, ServerlessSpec

pinecone = Pinecone(api_key="your-api-key")
index = pinecone.Index("trip-index")

In [ ]:
# 3. 검색 함수 정의

# 선호 숙소 형태 및 동행인 기반
def search_places(city, companions, lodging_style):
    query = f"Best accommdations in {city} for {companions} with focus on {lodging_style}."
    query_embedding = model.encode(query).tolist()
    namespace = f"{city}_lodging"
    results_style = index.query(vector=query_embedding, top_k=20, namespace=namespace, include_metadata=True)
    return results_style

In [ ]:
'''# 4. 숙소 리스트 생성을 위한 프롬프트 템플릿 - 한글 버전
prompt_template = """
당신은 숙소 추천 AI 전문가입니다. 아래의 정보를 바탕으로 {city}에서 {trip_duration} 동안의 여행 기간에 맞는 최적의 숙소 **5개**를 포함한 리스트를 생성하세요.

- 도시: {city}
- 여행 기간: {trip_duration}
- 여행 날짜: {travel_dates}
- 동행인: {companions}
- 숙소 스타일: {lodging_style}
- 1박 예산: {lodging_budget}

**제약 조건**:
- 숙소는 동행인의 취향과 숙소 스타일에 맞아야 합니다.
- 1박 예산을 초과하지 않도록 유의하세요.
- 각 숙소의 특징을 간결하게 서술하세요.
- 위치와 관련된 정보(주요 관광지와의 거리, 교통 편의성 등)를 포함하세요.
- 숙소는 {city}에서 안전한 지역에 위치해야 합니다.
- 리뷰 평점과 제공하는 주요 서비스(조식, 무료 와이파이, 주차, 수영장 등)를 포함하세요.
- 최대한 다양한 옵션(호텔, 아파트, 리조트 등)을 제시하세요.
- 추천 숙소 리스트:
{recommendations}

**추가 요청 사항**:
- 여행 날짜에 대해 추천하는 숙소의 예약 가능 여부와 체크인, 체크아웃 시간을 고려하세요.
"""
'''

'# 4. 숙소 리스트 생성을 위한 프롬프트 템플릿 - 한글 버전\nprompt_template = """\n당신은 숙소 추천 AI 전문가입니다. 아래의 정보를 바탕으로 {city}에서 {trip_duration} 동안의 여행 기간에 맞는 최적의 숙소 **5개**를 포함한 리스트를 생성하세요.\n\n- 도시: {city}\n- 여행 기간: {trip_duration}\n- 여행 날짜: {travel_dates}\n- 동행인: {companions}\n- 숙소 스타일: {lodging_style}\n- 1박 예산: {lodging_budget}\n\n**제약 조건**:\n- 숙소는 동행인의 취향과 숙소 스타일에 맞아야 합니다.\n- 1박 예산을 초과하지 않도록 유의하세요.\n- 각 숙소의 특징을 간결하게 서술하세요.\n- 위치와 관련된 정보(주요 관광지와의 거리, 교통 편의성 등)를 포함하세요.\n- 숙소는 {city}에서 안전한 지역에 위치해야 합니다.\n- 리뷰 평점과 제공하는 주요 서비스(조식, 무료 와이파이, 주차, 수영장 등)를 포함하세요.\n- 최대한 다양한 옵션(호텔, 아파트, 리조트 등)을 제시하세요.\n- 추천 숙소 리스트:\n{recommendations}\n\n**추가 요청 사항**:\n- 여행 날짜에 대해 추천하는 숙소의 예약 가능 여부와 체크인, 체크아웃 시간을 고려하세요.\n"""\n'

In [ ]:
# 4. 숙소 리스트 생성을 위한 프롬프트 템플릿 - 영어 버전
prompt_template = """
You are an AI expert in accommodation recommendations. Based on the information below, create a list of the **top 5** accommodations for a trip to {city} lasting {trip_duration}.

- City: {city}
- Duration of Stay: {trip_duration}
- Travel Dates: {travel_dates}
- Companions: {companions}
- Lodging Style: {lodging_style}
- Budget per Night: {lodging_budget}

**Requirements**:
- Accommodations should match the preferences of the companions and the lodging style.
- Ensure the budget per night is not exceeded.
- Provide a concise description of each accommodation’s features in one sentence.
- Include location details such as proximity to major attractions and transportation convenience.
- Ensure the accommodations are in safe areas of {city}.
- Include review ratings and key amenities (e.g., breakfast, free Wi-Fi, parking, swimming pool).
- Offer a variety of options (e.g., hotels, apartments, resorts).
- Recommended accommodation list:
{recommendations}

**Additional Requests**:
- Check availability for the travel dates and include check-in and check-out times if possible.
- Please answer in Korean.
"""

In [ ]:
# 5. 프롬프트와 LLMChain 설정
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4",
    openai_api_key = openai_api_key
)

# 숙소 추천 생성 함수
def generate_accommodation_recommendations(city, trip_duration, travel_dates, companions, lodging_style, lodging_budget, recommendations):
    # 템플릿에 사용자 정보 삽입
    formatted_prompt = prompt_template.format(
        city=city,
        trip_duration=trip_duration,
        travel_dates=travel_dates,
        companions=companions,
        lodging_style=lodging_style,
        lodging_budget=lodging_budget,
        recommendations=recommendations
    )

    # LangChain 프롬프트 구성
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template("You are an AI expert in accommodation recommendations."),
            HumanMessagePromptTemplate.from_template(formatted_prompt)
        ]
    )

    # LLMChain 설정 및 실행
    conversation = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True
    )

    # 결과 생성
    result = conversation.run({})
    return result


<ipython-input-9-8d75826cbd25>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [ ]:
# 6. 메인 함수: 사용자 입력 및 숙소 추천 실행
def main():
    # 사용자 입력 예시
    accommodation_details = {
        "city": "paris",
        "trip_duration": "three nights four days",
        "travel_dates": "2024-11-15 ~ 2024-11-18",
        "companions": "parents",
        "lodging_style": "luxury, swimming pool",
        "lodging_budget": "200euro"
    }

    # 파인콘에서 숙소 검색 실행 (상위 5개 숙소)
    search_results = search_places(
        city=accommodation_details["city"],
        companions=accommodation_details["companions"],
        lodging_style=accommodation_details["lodging_style"],
    )

    # 파인콘에서 가져온 추천 숙소 리스트 구성 (최대 20개)
    recommendations = "\n".join([
        f"- {match.metadata['1_이름']} (평점: {match.metadata['3_평점']}, 위치: {match.metadata['2_주소']}, 유형: {match.metadata['8_유형']})"
        for match in search_results.matches
    ])

    # 숙소 추천 생성 호출
    accommodation_recommendations = generate_accommodation_recommendations(
        city=accommodation_details["city"],
        trip_duration=accommodation_details["trip_duration"],
        travel_dates=accommodation_details["travel_dates"],
        companions=accommodation_details["companions"],
        lodging_style=accommodation_details["lodging_style"],
        lodging_budget=accommodation_details["lodging_budget"],
        recommendations=recommendations
    )

    # 결과 출력
    print(accommodation_recommendations)

In [ ]:
# 7. 코드 실행
if __name__ == "__main__":
    main()

<ipython-input-9-8d75826cbd25>:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(
<ipython-input-9-8d75826cbd25>:37: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation.run({})




> Entering new LLMChain chain...
Prompt after formatting:
System: You are an AI expert in accommodation recommendations.
Human: 
You are an AI expert in accommodation recommendations. Based on the information below, create a list of the **top 5** accommodations for a trip to paris lasting three nights four days.

- City: paris
- Duration of Stay: three nights four days
- Travel Dates: 2024-11-15 ~ 2024-11-18
- Companions: parents
- Lodging Style: luxury, swimming pool
- Budget per Night: 200euro

**Requirements**:
- Accommodations should match the preferences of the companions and the lodging style.
- Ensure the budget per night is not exceeded.
- Provide a concise description of each accommodation’s features in one sentence.
- Include location details such as proximity to major attractions and transportation convenience.
- Ensure the accommodations are in safe areas of paris.
- Include review ratings and key amenities (e.g., breakfast, free Wi-Fi, parking, swimming pool).
- Offer a 